## Chapter 03. Sentiment Analysis

#### with Tensorflow

In [ ]:
# !pip install tensorflow
# !pip install kiwipiepy

In [ ]:
# 데이터 다운로드
# !wget -c https://github.com/e9t/nsmc/raw/master/ratings_train.txt

In [2]:
import pandas as pd
nsmc = pd.read_csv('./data/ratings_train.txt', sep='\t')
nsmc.head()
# 품사 추출 계획

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
# 형태소 분석
from kiwipiepy import Kiwi

kiwi = Kiwi()

In [5]:
# 단어와 품사 추출
def extract_keywords(text):
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if pos[0] in 'NV':
            yield f'{token}/{pos}'

In [6]:
list(extract_keywords(nsmc.document[0]))

['더빙/NNG', '짜증/NNG', '나/VV', '목소리/NNG']

In [9]:
# 문서 단어 행렬
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import warnings
warnings.filterwarnings('ignore')

cv = CountVectorizer(max_features=1000, tokenizer=extract_keywords)
dtm = cv.fit_transform(nsmc.loc[0:1999, 'document'])

# 감성 분석 시, 두 가지 방식을 병행해보고 성능이 좋은 쪽 선택
trans = TfidfTransformer()
dtm2 = trans.fit_transform(dtm)

In [8]:
dtm.shape

(2000, 1000)

In [10]:
# 단어 목록
words = cv.get_feature_names_out()
words

array(['가/VV', '가/VX', '가능/NNG', '가볍/VA-I', '가슴/NNG', '가족/NNG', '가지/NNB',
       '가지/VV', '가치/NNG', '각본/NNG', '간/NNB', '갈등/NNG', '감독/NNG', '감동/NNG',
       '감사/NNG', '감상/NNG', '감성/NNG', '감정/NNG', '강요/NNG', '같/VA', '개/NNB',
       '개/NNG', '개그/NNG', '개그맨/NNG', '개념/NNG', '개봉/NNG', '개뿔/NNG',
       '개연/NNG', '개인/NNG', '개판/NNG', '거/NNB', '거기/NP', '거리/NNG', '거리/VV',
       '거부/NNG', '거슬리/VV', '거장/NNG', '걱정/NNG', '걸리/VV', '걸작/NNG',
       '검술/NNG', '것/NNB', '게이/NNG', '게임/NNG', '결국/NNG', '결말/NNG', '경/NNG',
       '고추/NNG', '곳/NNG', '곳곳/NNG', '공감/NNG', '공감/NNP', '공부/NNG',
       '공포/NNG', '공포물/NNG', '곽지민/NNP', '관객/NNG', '관계/NNG', '관람/NNG',
       '관심/NNG', '괜찮/VA', '교훈/NNG', '구경/NNG', '구리/VA', '구성/NNG', '구하/VV',
       '국민/NNG', '국산/NNG', '군/NNG', '군대/NNG', '굿/NNG', '궁금하/VA', '귀신/NNG',
       '귀엽/VA-I', '그/NP', '그거/NP', '그것/NP', '그녀/NP', '그래픽/NNG', '그러/VV',
       '그렇/VA', '그릇/NNG', '그리/VV', '그림/NNG', '그립/VA-I', '그지/NNG', '극/NNG',
       '극장/NNG', '극장판/NNG', '글/NNG', '급/NNG', '기/NNG', '기대/NNG'

In [11]:
# 저장
import joblib
joblib.dump({'words': words, 'dtm': dtm, 'dtm2': dtm2}, 'nsmc.pkl')

['nsmc.pkl']

#### 데이터 분할

In [12]:
# 불러오기
import joblib
data = joblib.load('nsmc.pkl')
locals().update(data)
# 지역 변수들을 사전 형태로 만들어줌 (data의 내용을 변수로 만들어줌)

In [13]:
dtm # words, dtm2 등도 가능

<2000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 12527 stored elements in Compressed Sparse Row format>

In [14]:
import pandas as pd
nsmc = pd.read_csv('./data/ratings_train.txt', sep='\t')

In [15]:
# 기계 학습을 위한 필수 전처리
type(nsmc.label.values) # 기존 Series 형식을 numpy array 형식으로 변경

numpy.ndarray

In [16]:
# 데이터 분할
from sklearn.model_selection import train_test_split
x = dtm # 단어 문서 행렬
y = nsmc.label.values[:2000]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1984)

#### 로지스틱 회귀 분석

In [17]:
# 모형 정의
import tensorflow as tf

model = tf.keras.models.Sequential()
# 모델의 구성요소를 순차적으로 삽입하는 모형
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
# Dense = 선형모형, 모든 종류의 입력에 일정한 가중치를 곱해줌
# 출력은 1개, input_shape 의 입력 형태는 생략, activation 정의를 통해 연속적인 예측을 하는 선형 회귀 모형이 아닌 로지스틱 회귀 모형으로 설정정

In [18]:
# 설정
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# optimizer = 모델의 학습의 방법, sgd (경사하강법) adam(최근 많이 사용하는 변형 형태)
# loss = 손실함수, binary_crossentropy (이항 교차 엔트로피 계산)
# metrics = 교차 엔트로피는 결과값을 보고 직관적인 이해가 힘드므로, 이해를 위한 보조 형태로 정확도 산출

In [19]:
# 훈련
model.fit(x_train.A, y_train, epochs=3)
# 현재 x_train은 압축되어 있는 형태이기 때문에 .A를 통해 압축을 풀어 입력
# epochs = 학습의 횟수

Epoch 1/3
50/50 [==============================] - 0s 1ms/step - loss: 0.6878 - accuracy: 0.5500
Epoch 2/3
50/50 [==============================] - 0s 989us/step - loss: 0.6711 - accuracy: 0.6519
Epoch 3/3
50/50 [==============================] - 0s 1ms/step - loss: 0.6559 - accuracy: 0.7169


In [20]:
# 테스트
model.evaluate(x_test.A, y_test)

13/13 [==============================] - 0s 1ms/step - loss: 0.6626 - accuracy: 0.6425


[0.6626058220863342, 0.6424999833106995]

In [21]:
# 저장
model.save('nsmc.krs')
# 폴더에 모델 형성에 필요한 데이터들이 저장됨됨

INFO:tensorflow:Assets written to: nsmc.krs\assets


INFO:tensorflow:Assets written to: nsmc.krs\assets


#### 가중치 분석

In [22]:
# 모형 불러오기
import tensorflow as tf
model = tf.keras.models.load_model('nsmc.krs') # 모델 저장 폴더 이름 호출

In [23]:
# 파라미터
w, b = model.weights # 모델에 입력되어 있는 여러 파라미터를 호출(함수 형태에서 호출 가능)
# w = 단어별 가중치 + 면 긍정, - 면 부정

In [24]:
# 단어별 가중치 표
import pandas as pd
word_sent = pd.DataFrame({'토큰': words, '가중치': w.numpy().flat})
# w 는 array 형태이기 때문에 데이터 프레임 형태로 만들어줌

In [25]:
# 부정단어
word_sent.sort_values('가중치').head()

,토큰,가중치
746,재미없/VA,-0.153808
599,없/VA,-0.144478
165,높/VA,-0.142886
539,쓰/VV,-0.138577
287,말/VX,-0.123461


In [26]:
# 긍정단어
word_sent.sort_values('가중치').tail()

,토큰,가중치
117,나/NP,0.114791
606,여운/NNG,0.119385
128,나이/NNG,0.121989
700,인상/NNG,0.122878
307,명작/NNG,0.143099


#### 희소 행렬 변환 (SparseTensor)

In [27]:
# 문서 단어 행렬을 압축된 형태로 학습시키는 방법
x_train
# 데이터가 용량이 클 경우, 압축을 풀 경우 학습이 진행이 안 될수도 있음
# 현재 scipy의 CSR 포맷이고 모델은 tensor flow 이기 때문에 형식이 맞지 않음

<1600x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 10015 stored elements in Compressed Sparse Row format>

In [28]:
# CSR 방식으로 압축된 문서 단어 행렬을 COO 방식으로 변환
x_coo = x_train.tocoo()

In [29]:
# 행 번호
x_coo.row

array([   0,    0,    0, ..., 1599, 1599, 1599])

In [30]:
# 열 번호
x_coo.col

array([683, 126,   0, ..., 465, 171, 375])

In [31]:
# 내용
x_coo.data

array([3, 1, 1, ..., 1, 1, 1], dtype=int64)

In [32]:
# 행 번호와 열 번호를 칼럼으로 합친다
import numpy as np
index = np.column_stack([x_coo.row, x_coo.col])
index

array([[   0,  683],
       [   0,  126],
       [   0,    0],
       ...,
       [1599,  465],
       [1599,  171],
       [1599,  375]])

In [33]:
# 텐서플로의 SparseTensor로 변환
x_train_sparse = tf.SparseTensor(index, x_coo.data, x_coo.shape)
x_train_sparse

SparseTensor(indices=tf.Tensor(
[[   0  683]
 [   0  126]
 [   0    0]
 ...
 [1599  465]
 [1599  171]
 [1599  375]], shape=(10015, 2), dtype=int64), values=tf.Tensor([3 1 1 ... 1 1 1], shape=(10015,), dtype=int64), dense_shape=tf.Tensor([1600 1000], shape=(2,), dtype=int64))

In [34]:
# 행과 열 번호를 정렬
x_train_sparse = tf.sparse.reorder(x_train_sparse)
x_train_sparse
# 행 번호와 열 번호의 순서가 섞여있을 경우, 모델에 입력할 수 없음

SparseTensor(indices=tf.Tensor(
[[   0    0]
 [   0  126]
 [   0  148]
 ...
 [1599  742]
 [1599  951]
 [1599  952]], shape=(10015, 2), dtype=int64), values=tf.Tensor([1 1 1 ... 1 1 7], shape=(10015,), dtype=int64), dense_shape=tf.Tensor([1600 1000], shape=(2,), dtype=int64))

In [35]:
# 모형 훈련에 사용
model.fit(x_train_sparse, y_train, epochs=1)

50/50 [==============================] - 0s 1ms/step - loss: 0.6422 - accuracy: 0.7406


#### Early Stopping

In [36]:
# validaiont_split 을 통해 중간 테스트에 사용할 validation data의 비율을 지정
# callbacks에서 EarlyStopping을 추가하면 각 에포크 끝마다 성능을 테스트하여 중단 여부 결정
model.fit(x_train.A, y_train, epochs=100, validation_split=0.1,
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy')])
# validation_loss가 더 이상 안 작아질 때 Early Stop
# monitor = 옵션 지정을 통해 해당 옵션이 발전하지 않을 때 학습 중단 결정

Epoch 1/100
45/45 [==============================] - 0s 4ms/step - loss: 0.6293 - accuracy: 0.7625 - val_loss: 0.6156 - val_accuracy: 0.8250
Epoch 2/100
45/45 [==============================] - 0s 2ms/step - loss: 0.6177 - accuracy: 0.7882 - val_loss: 0.6074 - val_accuracy: 0.8438
Epoch 3/100
45/45 [==============================] - 0s 2ms/step - loss: 0.6065 - accuracy: 0.8000 - val_loss: 0.5999 - val_accuracy: 0.8625
Epoch 4/100
45/45 [==============================] - 0s 2ms/step - loss: 0.5961 - accuracy: 0.8090 - val_loss: 0.5932 - val_accuracy: 0.8500


### Practice - kiwipiepy

In [37]:
from kiwipiepy import Kiwi
#형태소 분석기 초기화
kiwi = Kiwi()
text = '오늘은 자연어 처리를 배우기 좋은 날이다. 자연어 처리는 재미있다.'

In [38]:
result = kiwi.analyze(text)
result

[([Token(form='오늘', tag='NNG', start=0, len=2),
   Token(form='은', tag='JX', start=2, len=1),
   Token(form='자연어', tag='NNP', start=4, len=3),
   Token(form='처리', tag='NNG', start=8, len=2),
   Token(form='를', tag='JKO', start=10, len=1),
   Token(form='배우', tag='VV', start=12, len=2),
   Token(form='기', tag='ETN', start=14, len=1),
   Token(form='좋', tag='VA', start=16, len=1),
   Token(form='은', tag='ETM', start=17, len=1),
   Token(form='날', tag='NNG', start=19, len=1),
   Token(form='이', tag='VCP', start=20, len=1),
   Token(form='다', tag='EF', start=21, len=1),
   Token(form='.', tag='SF', start=22, len=1),
   Token(form='자', tag='IC', start=24, len=1),
   Token(form='연어', tag='NNG', start=25, len=2),
   Token(form='처리', tag='NNG', start=28, len=2),
   Token(form='는', tag='JX', start=30, len=1),
   Token(form='재미있', tag='VA', start=32, len=3),
   Token(form='다', tag='EF', start=35, len=1),
   Token(form='.', tag='SF', start=36, len=1)],
  -105.60535430908203)]

In [39]:
result[0][0] # result만 출력시, 형태소 분석의 점수가 같이 출력, [0]을 출력 시 점수가 가장 높은 형태서 분석 결과를 출력
# 형태소, 품사, 위치, 길이

[Token(form='오늘', tag='NNG', start=0, len=2),
 Token(form='은', tag='JX', start=2, len=1),
 Token(form='자연어', tag='NNP', start=4, len=3),
 Token(form='처리', tag='NNG', start=8, len=2),
 Token(form='를', tag='JKO', start=10, len=1),
 Token(form='배우', tag='VV', start=12, len=2),
 Token(form='기', tag='ETN', start=14, len=1),
 Token(form='좋', tag='VA', start=16, len=1),
 Token(form='은', tag='ETM', start=17, len=1),
 Token(form='날', tag='NNG', start=19, len=1),
 Token(form='이', tag='VCP', start=20, len=1),
 Token(form='다', tag='EF', start=21, len=1),
 Token(form='.', tag='SF', start=22, len=1),
 Token(form='자', tag='IC', start=24, len=1),
 Token(form='연어', tag='NNG', start=25, len=2),
 Token(form='처리', tag='NNG', start=28, len=2),
 Token(form='는', tag='JX', start=30, len=1),
 Token(form='재미있', tag='VA', start=32, len=3),
 Token(form='다', tag='EF', start=35, len=1),
 Token(form='.', tag='SF', start=36, len=1)]

In [40]:
# 명사만 추출하기
for token, pos, _, _ in result[0][0]:
    if pos.startswith('N'):
        print(token)
# for문 변수의 _ 처리는 받아서 사용하지 않는다면 _ 처리
# startswith = N으로 시작한다는 조건 부여 

오늘
자연어
처리
날
연어
처리


In [42]:
#명사 추출 함수
def extract_noun(text):
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if pos.startswith('N'):
            yield token
# if pos[0] in 'NV' ; 명사와 동사 동시 추출출명사 추출 함수
def extract_noun(text):
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
            if pos.startswith('N'):
                        yield token
                        # if pos[0] in 'NV' ; 명사와 동사 동시 추출출

In [43]:
list(extract_noun('어제는 홍차를 마시고, 오늘은 커피를 마셨다.'))

['어제', '홍차', '오늘', '커피']

#### Practice - Konlpy

In [ ]:
# !conda install -y -c conda-forge jpype1
# !pip install konlpy

In [44]:
# 최신 형태소 분석기인 Komoran 사용
from konlpy.tag import Komoran
# 형태소 분석기 초기화
tagger = Komoran()
text = '오늘은 자연어 처리를 배우기 좋은 날이다. 자연어 처리는 재미있다.'

In [45]:
tagger.pos(text)
# 형태소, 품사

[('오늘', 'NNG'),
 ('은', 'JX'),
 ('자연어', 'NNP'),
 ('처리', 'NNG'),
 ('를', 'JKO'),
 ('배우', 'VV'),
 ('기', 'ETN'),
 ('좋은 날', 'NNP'),
 ('이', 'VCP'),
 ('다', 'EF'),
 ('.', 'SF'),
 ('자연어', 'NNP'),
 ('처리', 'NNG'),
 ('는', 'JX'),
 ('재미있', 'VA'),
 ('다', 'EF'),
 ('.', 'SF')]

In [46]:
# 명사 추출 함수
tagger.nouns(text)

['오늘', '자연어', '처리', '좋은 날', '자연어', '처리']

#### Practice - stanza

In [ ]:
# !conda install -y -c pytorch pytorch torchvision
# !pip install stanza

In [47]:
import stanza
# stanza는 기계학습 방식을 사용하므로, 한국어 데이터 학습된 모형이 필요
# 모형 다운로드
stanza.download('ko')

INFO:stanza:Downloading default packages for language: ko (Korean) ...


INFO:stanza:Finished downloading models and saved to C:\Users\lucky\stanza_resources.


In [48]:
# 모형 호출
nlp = stanza.Pipeline('ko')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ko (Korean):
| Processor | Package        |
------------------------------
| tokenize  | kaist          |
| pos       | kaist_nocharlm |
| lemma     | kaist_nocharlm |
| depparse  | kaist_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [49]:
# 한국어 모형 'kaist'와 'gsd'가 있으며, 'kaist'가 기본
# 'gsd' 패키지 사용 용법
stanza.download('ko', package='gsd')
nlp = stanza.Pipeline('ko', package='gsd')

INFO:stanza:Downloading these customized packages for language: ko (Korean)...
| Processor | Package      |
----------------------------
| tokenize  | gsd          |
| pos       | gsd_nocharlm |
| lemma     | gsd_nocharlm |
| depparse  | gsd_nocharlm |
| pretrain  | conll17      |



INFO:stanza:File exists: C:\Users\lucky\stanza_resources\ko\pretrain\conll17.pt
INFO:stanza:Finished downloading models and saved to C:\Users\lucky\stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ko (Korean):
| Processor | Package      |
----------------------------
| tokenize  | gsd          |
| pos       | gsd_nocharlm |
| lemma     | gsd_nocharlm |
| depparse  | gsd_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [50]:
text = '오늘은 자연어 처리를 배우기 좋은 날이다. 자연어 처리는 재미있다.'

In [51]:
# 문장 형태소 분석 후 doc 변수에 저장
doc = nlp(text)
doc

[
  [
    {
      "id": 1,
      "text": "오늘은",
      "lemma": "오늘+은",
      "upos": "NOUN",
      "xpos": "NNG+JX",
      "head": 6,
      "deprel": "nsubj",
      "start_char": 0,
      "end_char": 3
    },
    {
      "id": 2,
      "text": "자연어",
      "lemma": "자연어",
      "upos": "NOUN",
      "xpos": "NNG",
      "head": 4,
      "deprel": "obj",
      "start_char": 4,
      "end_char": 7
    },
    {
      "id": 3,
      "text": "처리를",
      "lemma": "처리+를",
      "upos": "NOUN",
      "xpos": "NNG+JKO",
      "head": 2,
      "deprel": "flat",
      "start_char": 8,
      "end_char": 11
    },
    {
      "id": 4,
      "text": "배우기",
      "lemma": "배우+기",
      "upos": "NOUN",
      "xpos": "VV+ETN",
      "head": 5,
      "deprel": "nsubj",
      "start_char": 12,
      "end_char": 15
    },
    {
      "id": 5,
      "text": "좋은",
      "lemma": "좋+은",
      "upos": "VERB",
      "xpos": "VA+ETM",
      "head": 6,
      "deprel": "acl:relcl",
      "start_char": 16,
      

In [52]:
# .sentences에 분석된 문장들이 리스트 형태로 저장
sentence = doc.sentences[0]
sentence

[
  {
    "id": 1,
    "text": "오늘은",
    "lemma": "오늘+은",
    "upos": "NOUN",
    "xpos": "NNG+JX",
    "head": 6,
    "deprel": "nsubj",
    "start_char": 0,
    "end_char": 3
  },
  {
    "id": 2,
    "text": "자연어",
    "lemma": "자연어",
    "upos": "NOUN",
    "xpos": "NNG",
    "head": 4,
    "deprel": "obj",
    "start_char": 4,
    "end_char": 7
  },
  {
    "id": 3,
    "text": "처리를",
    "lemma": "처리+를",
    "upos": "NOUN",
    "xpos": "NNG+JKO",
    "head": 2,
    "deprel": "flat",
    "start_char": 8,
    "end_char": 11
  },
  {
    "id": 4,
    "text": "배우기",
    "lemma": "배우+기",
    "upos": "NOUN",
    "xpos": "VV+ETN",
    "head": 5,
    "deprel": "nsubj",
    "start_char": 12,
    "end_char": 15
  },
  {
    "id": 5,
    "text": "좋은",
    "lemma": "좋+은",
    "upos": "VERB",
    "xpos": "VA+ETM",
    "head": 6,
    "deprel": "acl:relcl",
    "start_char": 16,
    "end_char": 18
  },
  {
    "id": 6,
    "text": "날이다",
    "lemma": "날+이+다",
    "upos": "VERB",
    "xpos": "N

In [53]:
# .words에 분석된 어절들이 리스트 형태로 저장
word = sentence.words[0]
word

{
  "id": 1,
  "text": "오늘은",
  "lemma": "오늘+은",
  "upos": "NOUN",
  "xpos": "NNG+JX",
  "head": 6,
  "deprel": "nsubj",
  "start_char": 0,
  "end_char": 3
}

In [54]:
# 어절을 이루는 형태소의 표제어들을 보여줌; 형태소가 2개 이상인 경우 +표시로 구분 어절을 이루는 형태소의 표제어들을 보여줌; 형태소가 2개 이상인 경우 +표시로 구분
word.lemma

'오늘+은'

In [55]:
# 품사 정보
word.xpos

'NNG+JX'

#### Practice - 단어와 품사 태그 짝짓기

In [56]:
# stanza 는 한 어절에 형태소가 여러 개 있으면, 표제어와 품사가 +로 묶여서 나온다는 불편함이 존재; 해당 불편함을 해소하는 과정

#import stanza
nlp = stanza.Pipeline('ko')
text = '오늘은 자연어 처리를 공부하기 좋은 날이다.'

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ko (Korean):
| Processor | Package        |
------------------------------
| tokenize  | kaist          |
| pos       | kaist_nocharlm |
| lemma     | kaist_nocharlm |
| depparse  | kaist_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [57]:
doc = nlp(text)

In [58]:
word = doc.sentences[0].words[0]
word

{
  "id": 1,
  "text": "오늘은",
  "lemma": "오늘+은",
  "upos": "NOUN",
  "xpos": "ncn+jxt",
  "head": 6,
  "deprel": "dislocated",
  "start_char": 0,
  "end_char": 3
}

In [59]:
word.lemma

'오늘+은'

In [60]:
word.xpos

'ncn+jxt'

In [61]:
# + 을 기준으로 분리
word.lemma.split('+')

['오늘', '은']

In [62]:
word.xpos.split('+')

['ncn', 'jxt']

In [63]:
# zip 함수를 통해, 각 리스트의 번호끼리 짝지어 리스트로 반환
list(zip(word.lemma.split('+'), word.xpos.split('+')))

[('오늘', 'ncn'), ('은', 'jxt')]

In [64]:
for word in doc.sentences[0].words:
    lemma = word.lemma.split('+')
    xpos = word.xpos.split('+')
    for token, pos in zip(lemma, xpos):
        print(token, pos)

오늘 ncn
은 jxt
자연어 ncn
처리 ncpa
를 jco
공부 ncpa
하 xsv
기 etn
좋 paa
ㄴ etm
날 ncn
이 jp
다 ef
. sf


In [65]:
# 명사 추출
text = '오늘 커피를 마셨다.'
doc = nlp(text)

In [66]:
# 명사 표시는 n 으로 시작하므로, 해당하는 형태소만 출력 명사 표시는 n 으로 시작하므로, 해당하는 형태소만 출력
for word in doc.sentences[0].words:
    lemma = word.lemma.split('+')
    xpos = word.xpos.split('+')

    for tok, pos in zip(lemma, xpos):
        if pos.startswith('n'):
            print(tok)

오늘
커피


In [67]:
def extract_noun(text):
    doc = nlp(text)
    for sentence in doc.sentences:
        for word in sentence.words:
            lemma = word.lemma.split('+')
            xpos = word.xpos.split('+')
            for tok, pos in zip(lemma, xpos):
                if pos.startswith('n'):
                    yield tok
# 결과가 나올 때마다(명사를 하나 만날 때마다) 반환; yield
# list 로 묶어주면 결과값 출력
# 빈 리스트를 만들어서 tok를 append 해주고, 마지막에 return 해주는 식으로도 함수 작성 가능

In [68]:
list(extract_noun('편의점에서 커피를 샀다.'))

['편의점', '커피']

In [69]:
list(extract_noun('토끼는 당근을 좋아할까?'))

['토끼', '당근']

#### Practice - 한국어 단어 문서 행렬

In [70]:
# 데이터 다운로드
import requests

res = requests.get('https://github.com/e9t/nsmc/raw/master/ratings_train.txt')

with open('nsmc_train.csv', 'wb') as f:
    f.write(res.content)

In [71]:
# 데이터 불러오기
import pandas as pd
nsmc = pd.read_csv('nsmc_train.csv', sep='\t')
nsmc.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [72]:
# 명사 추출 함수
def extract_nouns(text):
    doc = nlp(text)
    for sentence in doc.sentences:
        for word in sentence.words:
            lemma = word.lemma.split('+')
            xpos = word.xpos.split('+')
            for lem, pos in zip(lemma, xpos):
                if pos.startswith('n'):
                    yield lem

In [73]:
nsmc.loc[3, 'document']

'교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'

In [74]:
list(extract_nouns(nsmc.loc[3, 'document']))

['교도소', '이야기', '재미', '평점', '조정']

In [75]:
# TDM 만들기
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=100, tokenizer=extract_nouns)
# 추출할 단어 갯수 100개 설정, 빈도 순으로 최대 100 단어까지 포함, tokenizer 지정을 통해 한국어 명사만 추출해 TDM 생성

In [76]:
# 101개의 영화평 분석 후 결과 tdm에 저장; 실습을 위해 적은 데이터만 사용
tdm = cv.fit_transform(nsmc.loc[0:100, 'document'])
tdm.shape

(101, 100)

In [78]:
word_count = pd.DataFrame({
    '단어': cv.get_feature_names_out(),
    '빈도': tdm.sum(axis=0).flat
})

In [79]:
word_count.sort_values('빈도', ascending=False).head()

,단어,빈도
50,영화,21
48,연기,7
98,평점,5
12,내용,5
91,진짜,4


In [80]:
# 빈도표 저장
word_count.to_excel('nsmc-count.xlsx')

In [81]:
# 한국어 단어 구름
import pandas as pd
word_count = pd.read_excel('nsmc-count.xlsx')
word_count

,Unnamed: 0,단어,빈도
0,0,1,2
1,1,10,2
2,2,3,2
3,3,8,2
4,4,ㅋㅋㅋ,2
...,...,...,...
95,95,최고,4
96,96,캐릭터,2
97,97,평범,3
98,98,평점,5


In [82]:
# 표를 사전 형태로 변환 표를 사전 형태로 변환
count_dic = word_count.set_index('단어')['빈도'].to_dict()
count_dic

{'1': 2,
 '10': 2,
 '3': 2,
 '8': 2,
 'ㅋㅋㅋ': 2,
 '감동': 2,
 '것': 3,
 '관객': 2,
 '그것': 2,
 '기대': 2,
 '나': 2,
 '내': 2,
 '내용': 5,
 '년대': 2,
 '느끼': 2,
 '드라마': 3,
 '듯': 2,
 '만찬': 2,
 '물건': 2,
 '뭐': 2,
 '바베트': 2,
 '사람': 4,
 '사랑': 2,
 '생각': 2,
 '속': 2,
 '수': 2,
 '스토리': 2,
 '신문': 1,
 '신선': 1,
 '신카이': 1,
 '실력': 1,
 '실망': 1,
 '심심': 1,
 '심오': 1,
 '아..시간아': 1,
 '아쉽네요ㅠㅠ': 1,
 '아예안나오': 1,
 '아이돌': 1,
 '아햏햏': 3,
 '악역': 1,
 '안': 3,
 '안나': 1,
 '억지': 1,
 '언제': 2,
 '엄포스': 1,
 '엉망': 1,
 '에속': 1,
 '에피소드': 2,
 '연기': 7,
 '영혼': 1,
 '영화': 21,
 '영화!스파이더맨맨': 1,
 '예전': 1,
 '예측': 1,
 '오버연기': 1,
 '온몸': 1,
 '완전': 4,
 '윈투어': 1,
 '유령': 1,
 '윤제문': 1,
 '은은하': 1,
 '은희': 1,
 '음식': 3,
 '음악': 1,
 '의학상': 1,
 '이': 2,
 '이거': 4,
 '이드라마': 1,
 '이민기': 2,
 '이범수': 1,
 '이야기': 2,
 '이응경': 1,
 '이정': 1,
 '이틀': 1,
 '이해': 1,
 '익살': 1,
 '인것': 1,
 '인상': 1,
 '일': 1,
 '일본': 1,
 '자체': 2,
 '재미': 2,
 '전개': 3,
 '절대': 3,
 '점': 4,
 '정신': 4,
 '조금': 3,
 '조작': 2,
 '죄인': 2,
 '줄': 2,
 '지루': 2,
 '진짜': 4,
 '짜증': 2,
 '차': 2,
 '초반': 2,
 '최고': 4,
 '캐릭터': 2,
 '평범

In [91]:
from wordcloud import WordCloud
# Windows 용 글꼴 지정
wc = WordCloud(font_path='C:/Windows/Fonts/NanumBarunGothic.ttf',
               background_color='white',
               max_words=50,
               width=400, height=300)

In [84]:
# !apt install fonts-nanum # Colab 용 글꼴 설치
# Colab 용
wc = WordCloud(font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
               background_color='white',
               max_words=50,
               width=400, height=300)

In [ ]:
cloud = wc.fit_words(count_dic)
cloud.to_image()